# 参数服务器
:label:`sec_parameterserver`

当我们从单个GPU扩展到多个GPU，再到包含多个GPU的多台服务器，甚至这些设备可能分布在多个机架和网络交换机上时，我们的分布式和并行训练算法需要变得更加复杂。细节非常重要，因为不同的互连具有非常不同的带宽（例如，在适当的设置下，NVLink可以提供高达100 GB/s的带宽（通过6条链路），PCIe 4.0（16通道）提供32 GB/s的带宽，而即使是高速100GbE以太网也只有10 GB/s）。同时，期望一个统计建模者成为网络和系统专家是不合理的。

参数服务器的核心思想在:citet:`Smola.Narayanamurthy.2010`中首次提出，用于分布式潜在变量模型的背景下。随后在:citet:`Ahmed.Aly.Gonzalez.ea.2012`中描述了推送和拉取语义，而在:citet:`Li.Andersen.Park.ea.2014`中则描述了系统及开源库。接下来我们将讨论为提高效率所需的各种组件。

## 数据并行训练

让我们回顾一下分布式训练中的数据并行训练方法。由于它在实际实现中显著更简单，因此在本节中我们将仅使用这种方法。除了图上的深度学习之外，几乎没有任何其他用例会偏好其他并行策略，因为如今的GPU拥有充足的内存。:numref:`fig_parameterserver`展示了我们在:numref:`sec_multi_gpu`中实现的数据并行变体。其中的关键在于梯度聚合发生在单一GPU（GPU 0）上，然后更新后的参数被重新广播给所有GPU。

![左：单GPU训练。右：一种多GPU训练变体：(1) 计算损失和梯度，(2) 所有梯度在一个GPU上聚合，(3) 发生参数更新并将参数重新分配给所有GPU。](../img/ps.svg)
:label:`fig_parameterserver`

回顾来看，选择在GPU 0上聚合梯度似乎有些随意。毕竟，我们也可以选择在CPU上进行聚合。实际上，我们甚至可以选择将部分参数在某一个GPU上聚合，而另一些在另一个GPU上聚合。只要优化算法支持这样做，实际上没有理由不能这样做。例如，如果我们有四个参数向量及其相关梯度$\mathbf{g}_1, \ldots, \mathbf{g}_4$，我们可以对每个$\mathbf{g}_i$（$i = 1, \ldots, 4$）在不同GPU上分别进行梯度聚合。

这种推理看起来有些任意且轻率。毕竟，数学原理在整个过程中是一样的。然而，我们处理的是真实的物理硬件，其中不同的总线有不同的带宽，如在:numref:`sec_hardware`中讨论的那样。考虑一个真正的4路GPU服务器，如:numref:`fig_bw_hierarchy`所示。如果连接特别好，可能会有一个100 GbE网络卡。更常见的数字是在1-10 GbE范围内，有效带宽为100 MB/s至1 GB/s。由于CPU的PCIe通道太少，无法直接连接所有GPU（例如，消费级Intel CPU有24个通道），我们需要一个多路复用器。CPU通过16x Gen3链接的带宽为16 GB/s。这也是每个GPU连接到交换机的速度。这意味着设备间通信更为高效。

![一个4路GPU服务器。](../img/bw-hierarchy.svg)
:label:`fig_bw_hierarchy`

为了便于讨论，假设梯度大小为160 MB。在这种情况下，将梯度从剩余的3个GPU发送到第4个GPU需要30毫秒（每次传输需要10毫秒 = 160 MB / 16 GB/s）。再加上30毫秒来将权重向量发回，总共需要60毫秒。如果我们把所有数据都发送到CPU，则会增加40毫秒的延迟，因为*每个* GPU都需要将数据发送到CPU，总计80毫秒。最后假设我们可以将梯度分成4个部分，每部分40 MB。现在我们可以同时在不同GPU上对每一部分进行聚合，因为PCIe交换机提供了全带宽操作。这将时间从30毫秒缩短到了7.5毫秒，使得同步操作总共只需15毫秒。简而言之，根据我们如何同步参数，相同的操作可以花费15毫秒到80毫秒不等。:numref:`fig_ps_distributed`描绘了参数交换的不同策略。

![参数同步策略。](../img/ps-distributed.svg)
:label:`fig_ps_distributed`

请注意，当涉及到性能提升时，我们还有另一个工具可供使用：在深层网络中，从顶部到底部计算所有梯度需要一些时间。即使我们还在忙于为其他参数组计算梯度，也可以开始同步某些参数组的梯度。有关如何在[Horovod](https://github.com/horovod/horovod)中执行此操作的详细信息，请参见例如:citet:`Sergeev.Del-Balso.2018`。

## 环形同步

当涉及到现代深度学习硬件上的同步时，我们经常遇到高度定制化的网络连接。例如，AWS p3.16xlarge和NVIDIA DGX-2实例共享:numref:`fig_nvlink`所示的连接结构。每个GPU通过一条PCIe链路连接到主机CPU，该链路最佳运行速度为16 GB/s。此外，每个GPU还有6个NVLink连接，每个连接能够双向传输300 Gbit/s。这相当于大约每条链路每方向18 GB/s。简而言之，总的NVLink带宽明显高于PCIe带宽。问题是如何最有效地利用它。

![8 V100 GPU服务器上的NVLink连接（图片来自NVIDIA）。](../img/nvlink.svg)
:label:`fig_nvlink`

事实证明，最优的同步策略是将网络分解成两个环，并使用它们直接同步数据 :cite:`Wang.Li.Liberty.ea.2018`。:numref:`fig_nvlink_twoloop`说明了网络可以被分解成一个双倍NVLink带宽的环（1-2-3-4-5-6-7-8-1）和一个常规带宽的环（1-4-6-3-5-8-2-7-1）。在这种情况下设计高效的同步协议并不简单。

![将NVLink网络分解为两个环。](../img/nvlink-twoloop.svg)
:label:`fig_nvlink_twoloop`

考虑以下思想实验：给定一个由$n$个计算节点（或GPU）组成的环，我们可以将梯度从第一个节点发送到第二个节点。在那里，它被添加到本地梯度中，然后发送到第三个节点，依此类推。经过$n-1$步后，可以在最后一个访问的节点中找到汇总梯度。也就是说，梯度聚合的时间随着节点数量线性增长。但如果这样操作，算法效率相当低。毕竟，任何时候只有一个节点在通信。如果我们把梯度分成$n$块，从节点$i$开始同步第$i$块呢？由于每块大小为$1/n$，总时间现在变为$(n-1)/n \approx 1$。换句话说，随着环的大小增加，梯度聚合所花费的时间*不会增长*。这是一个相当惊人的结果。:numref:`fig_ringsync`说明了在$n=4$个节点上的步骤序列。

![跨4个节点的环形同步。每个节点开始将其部分梯度传输给其左侧邻居，直到组装好的梯度可以在其右侧邻居中找到。](../img/ringsync.svg)
:label:`fig_ringsync`

如果我们使用相同的示例，在8个V100 GPU之间同步160 MB，我们得到的大约时间为$2 \cdot 160 \textrm{MB} / (3 \cdot 18 \textrm{GB/s}) \approx 6 \textrm{ms}$。这比使用PCIe总线要好，尽管我们现在使用的是8个GPU。需要注意的是，在实践中这些数字会稍差一些，因为深度学习框架通常无法将通信组合成大的突发传输。

需要注意的是，有一种普遍的误解认为环形同步与其他同步算法从根本上不同。唯一的区别是同步路径与简单的树相比稍微复杂一些。

## 多机训练

在多台机器上进行分布式训练增加了另一个挑战：我们需要通过相对较低带宽的网络进行通信，这种网络在某些情况下可能慢一个数量级以上。
跨设备同步很棘手。毕竟，运行训练代码的不同机器会有细微的速度差异。因此，如果我们想要使用同步的分布式优化，就需要*同步*它们。:numref:`fig_ps_multimachine`展示了分布式并行训练是如何发生的。

1. 在每台机器上读取一批（不同的）数据，将其拆分到多个GPU上并传输到GPU内存中。在每个GPU批上单独计算预测值和梯度。
2. 将所有本地GPU上的梯度聚合到一个GPU上（或者在不同GPU上对部分梯度进行聚合）。
3. 将梯度发送到CPU。
4. CPU将梯度发送到中央参数服务器，该服务器聚合所有梯度。
5. 使用聚合梯度更新参数，并将更新后的参数广播回各个CPU。
6. 信息被发送到一个（或多个）GPU。
7. 更新后的参数被分布到所有GPU上。

![多机多GPU分布式并行训练。](../img/ps-multimachine.svg)
:label:`fig_ps_multimachine`

这些操作中的每一个看起来都相当直接。确实，它们可以在单台机器内高效地完成。但一旦我们看多台机器的情况，就可以看到中央参数服务器成为了瓶颈。毕竟，每台服务器的带宽是有限的，因此对于$m$个工作节点来说，将所有梯度发送到服务器所需的时间为$\mathcal{O}(m)$。我们可以通过增加服务器的数量到$n$来突破这个障碍。此时，每台服务器只需要存储$\mathcal{O}(1/n)$的参数，因此更新和优化的总时间为$\mathcal{O}(m/n)$。
匹配这两个数字可以实现无论多少工作节点都能保持恒定的扩展性。实际上，我们使用*相同*的机器作为工作节点和服务器。:numref:`fig_ps_multips`展示了这种设计（详情请参阅:cite:`Li.Andersen.Park.ea.2014`）。
特别是，确保多台机器协同工作而不产生不合理延迟是非常复杂的。

![上：单个参数服务器是瓶颈，因为它的带宽是有限的。下：多个参数服务器存储部分参数，具有聚合带宽。](../img/ps-multips.svg)
:label:`fig_ps_multips`

## 键值存储

在实践中实现分布式多GPU训练所需的步骤并不简单。
这就是为什么使用一个通用抽象——即重新定义更新语义的*键值存储*是有益的原因。

在许多工作节点和许多GPU上，梯度$i$的计算可以定义为

$$\mathbf{g}_{i} = \sum_{k \in \textrm{workers}} \sum_{j \in \textrm{GPUs}} \mathbf{g}_{ijk},$$

其中$\mathbf{g}_{ijk}$是工作节点$k$上的GPU$j$分割出的梯度$i$的一部分。
这一操作的关键在于它是*可交换归约*，即将许多向量变成一个，并且应用操作的顺序无关紧要。这对于我们的目的非常有利，因为我们不需要（也不必）对哪个梯度何时收到进行细粒度控制。此外，注意这个操作在不同的$i$之间是独立的。

这允许我们定义以下两种操作：*push*，累积梯度；*pull*，检索聚合梯度。由于我们有许多不同的梯度集（毕竟我们有很多层），我们需要用键$i$来索引梯度。与Dynamo引入的键值存储:cite:`DeCandia.Hastorun.Jampani.ea.2007`相似并非巧合。它们也满足许多类似特征，特别是在将参数分布在多个服务器上时。

键值存储的push和pull操作如下描述：

* **push(key, value)** 将特定梯度（值）从工作节点发送到公共存储。在那里，值会被聚合，例如通过求和。
* **pull(key, value)** 从公共存储中检索聚合值，例如在合并所有工作节点的梯度之后。

通过将关于同步的所有复杂性隐藏在简单的push和pull操作后面，我们可以解耦希望用简单术语表达优化的统计建模者的需求与需要处理分布式同步内在复杂性的系统工程师的需求。

## 总结

* 同步需要高度适应特定的网络基础设施和服务器内的连接。这可以显著影响同步所需的时间。
* 对于p3和DGX-2服务器，环形同步可能是最优的。对于其他服务器可能不是如此。
* 当增加多个参数服务器以提高带宽时，分层同步策略效果很好。

## 练习

1. 你能否进一步增加环形同步的效率？提示：你可以双向发送消息。
1. 是否允许异步通信（同时计算仍在进行）？这对性能有何影响？
1. 如果在长时间运行的计算过程中丢失了一台服务器怎么办？我们如何设计一个*容错*机制以避免完全重启计算？

[讨论](https://discuss.d2l.ai/t/366)